# CTHD August Sales Analysis

## Preface

## Data Cleaning

## Data Exploratory

## Visualization

# Data Cleaning

## Original File: cthd_report_aug.csv

In [2]:
# Import the modules
import pandas as pd
import numpy as np

In [3]:
# the main data we are gonna use
# Original csv file that Ian provided: 臥虎藏龍 Crouch & Hide _ Tiger Dragon Bar月報.csv
# renamed it to cthd_report_aug.csv

file_path = "/Users/lucaslee/Documents/GitHub/Lucas-Data-Scientist-Lab/data_cthd_Aug/cthd_report_aug.csv"


In [4]:
df_raw = pd.read_csv(file_path, skiprows=1)

# print original columns:
print(df_raw.columns)
# result: ['銷售日期', '星期', '銷售總額', '折扣金額', '營業額', '稅額', '營業淨額', '客數', '周轉率', '人均價', '交易數', '交易均價']

Index(['銷售日期', '星期', '銷售總額', '折扣金額', '營業額', '稅額', '營業淨額', '客數', '周轉率', '人均價',
       '交易數', '交易均價'],
      dtype='object')


In [5]:
# Only select the rows with aug info in:
df = df_raw.loc[df_raw['銷售日期'].str.contains('2022-08')]

# Select columns that we are interested
df = df[[
    '銷售日期', '星期', '銷售總額', '折扣金額', '營業額', '客數', '人均價', '交易數', '交易均價'
]]

# Rename the column to English
df.columns = [
    'sales_date', 'weekday', 'sales_total', 'sales_discount', 'sales_result', 'guest_num', 'price_per_guest', 'orders_num', 'price_per_order'
]

In [6]:
df['sales_total'] = df['sales_total'].str.replace(",", "").astype(int)
df['sales_discount'] = df['sales_discount'].str.replace(",", "").astype(int)
df['sales_result'] = df['sales_result'].str.replace(",", "").astype(int)
df['guest_num'] = df['guest_num'].str.replace(",", "").astype(int)
df['price_per_guest'] = df['price_per_guest'].str.replace(",", "").astype(int)
df['price_per_order'] = df['price_per_order'].str.replace(",", "").astype(int)

In [7]:
# weekday transform: using np.select
condlist = [
    df['weekday'] == '一',
    df['weekday'] == '二',
    df['weekday'] == '三',
    df['weekday'] == '四',
    df['weekday'] == '五',
    df['weekday'] == '六',
    df['weekday'] == '日',
]
choicelist = [
    'Mon',
    'Tue',
    'Wed',
    'Thu',
    'Fri',
    'Sat',
    'Sun'
]

df['weekday'] = np.select(condlist, choicelist, 'Unknown')


In [8]:
# sales_date transform to datetime
df['sales_date'] = pd.to_datetime(df.sales_date)

In [9]:
# weekday vs weekend
weekday_list = [
    'Mon',
    'Tue',
    'Wed',
    'Thu',
]
weekend_list = [
    'Fri', 'Sat', 'Sun'
]

# Using np.select()
condlist = [df['weekday'].isin(weekday_list), df['weekday'].isin(weekend_list)]
choicelist = ['weekday', 'weekend']
df['weekend_label'] = np.select(condlist, choicelist, 'Unknown')


df.pivot_table(index=['weekend_label'],
               values=['sales_result', 'sales_discount', 'guest_num'],
               aggfunc='mean')

,guest_num,sales_discount,sales_result
weekend_label,,,
weekday,29.473684,434.894737,28873.736842
weekend,62.333333,929.916667,64353.250000


In [10]:
df.to_csv('cthd_report_aug_transform.csv', index=False)

## Original File: 點餐明細(1)

In [11]:
# 使用檔案: 點餐明細(1)
file_path = '/Users/lucaslee/Documents/GitHub/Lucas-Data-Scientist-Lab/data_cthd_Aug/點餐明細 (1).csv'

df_detail = pd.read_csv(file_path)
df_detail.columns

Index(['交易序號', '發票編號', '類別名稱', '商品名稱', '數量', '點單金額', '客數', '商品單位'], dtype='object')

In [12]:
column_list = ['交易序號', '類別名稱', '商品名稱', '數量', '點單金額']

df_detail = df_detail[column_list]

# chi2eng rename the columns
df_detail.columns = ['sales_date', 'product_type', 'product_name', 'amount', 'price']

In [13]:
# only want the first 8 char of the sales_date data and transform to datetime
df_detail['sales_date'] = pd.to_datetime(df_detail.sales_date.apply(lambda x: x[:8]))

In [14]:
df_detail.to_csv('cthd_sales_detail_aug_transform.csv', index=False)

## 交易明細副檔20220906.csv

In [34]:
file_path = '/Users/lucaslee/Documents/GitHub/Lucas-Data-Scientist-Lab/data_cthd_Aug/交易明細副檔20220906.csv'

df_order_detail_raw = pd.read_csv(file_path)
df_order_detail_raw.head()

,分店代碼,分店名稱,建立時間,單號,發票號碼,商品編號,商品名稱,標籤名稱,標籤價錢,商品單價,...,Martini 橄欖檸檬皮,清酒,點單隻送,情人節套餐加購,杯子數量,Honey Lemon,Hot Tea,Shot杯種類,破杯,破杯原因
0,1.0,臥虎藏龍 Crouch & Hide : Tiger Dragon Bar,2022-08-15 19:59:55,6,NaN,NaN,Dry Martini,With Lemon Peels,0.0,320.0,...,With Lemon Peels,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,臥虎藏龍 Crouch & Hide : Tiger Dragon Bar,2022-08-05 23:47:35,3,NaN,NaN,Earl Grey Milk Tea I/H,正常,0.0,220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,臥虎藏龍 Crouch & Hide : Tiger Dragon Bar,2022-08-05 23:47:35,3,NaN,NaN,Earl Grey Milk Tea I/H,正常,0.0,220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,臥虎藏龍 Crouch & Hide : Tiger Dragon Bar,2022-08-05 23:47:35,3,NaN,NaN,松阪豬炒水蓮,NaN,0.0,220.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,臥虎藏龍 Crouch & Hide : Tiger Dragon Bar,2022-08-05 23:47:35,3,NaN,NaN,轟炸雞軟骨,NaN,0.0,240.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
print(df_order_detail_raw.columns)

column_interested_list = ['建立時間', '單號', '商品名稱', '標籤名稱', '標籤價錢',
       '商品單價', '數量', '銷售金額', '單品折扣總額', '折扣名稱', '折扣總額', '實收金額', '淨額', '商品分類']

df_order_detail = df_order_detail_raw[column_interested_list]
df_order_detail.head()

Index(['分店代碼', '分店名稱', '建立時間', '單號', '發票號碼', '商品編號', '商品名稱', '標籤名稱', '標籤價錢',
       '商品單價', '數量', '銷售金額', '單品折扣總額', '折扣名稱', '折扣總額', '實收金額', '淨額', '商品分類',
       '用餐方式', '訂單來源', '冰塊與否', 'On the Rocks', 'Neat', '經典調酒', 'Shot種類', '服務鈴',
       '軟飲冰塊', '補單', '濃度', '香菜與否', '蔥與否', '店員名稱', '小黃瓜', 'Martini 橄欖檸檬皮', '清酒',
       '點單隻送', '情人節套餐加購', '杯子數量', 'Honey Lemon', 'Hot Tea', 'Shot杯種類', '破杯',
       '破杯原因'],
      dtype='object')


,建立時間,單號,商品名稱,標籤名稱,標籤價錢,商品單價,數量,銷售金額,單品折扣總額,折扣名稱,折扣總額,實收金額,淨額,商品分類
0,2022-08-15 19:59:55,6,Dry Martini,With Lemon Peels,0.0,320.0,1.0,320,0.0,NaN,0,320,305.0,Other Classic
1,2022-08-05 23:47:35,3,Earl Grey Milk Tea I/H,正常,0.0,220.0,1.0,220,0.0,NaN,0,220,210.0,Soft Drink
2,2022-08-05 23:47:35,3,Earl Grey Milk Tea I/H,正常,0.0,220.0,1.0,220,0.0,NaN,0,220,210.0,Soft Drink
3,2022-08-05 23:47:35,3,松阪豬炒水蓮,NaN,0.0,220.0,1.0,220,0.0,NaN,0,220,210.0,下酒菜
4,2022-08-05 23:47:35,3,轟炸雞軟骨,NaN,0.0,240.0,1.0,240,0.0,NaN,0,240,229.0,炸物


In [36]:
# drop na value if created date is na
condition = ~(df_order_detail['建立時間'].isna())
df_order_detail = df_order_detail[condition]

In [37]:
# dig deeper of the cocktail and wine type
# check the unique value of different product type
df_order_detail.商品分類.unique()
alc_list = ['Signature Cocktail', 'Classic Cocktail', 'Other Classic', 'Beer', 'Single Malt', 'Shot', 'House Wine', '燒酒', 'Gin', 'Champagne', 'Blended', 'Cognac', 'Bourbon']

df_alc = df_order_detail[df_order_detail.商品分類.isin(alc_list)]

# rename columns
df_alc.columns = ['sales_date', 'order_num', 'product_name', 'label_name', 'label_price', 'product_price', 'product_amount', 'sales_price', 'discount', 'discount_name', 'discount_total', 'real_receive_price', 'price_net', 'product_type']

In [38]:
# df_alc.head()
df_alc_clean = df_alc[['sales_date', 'product_type', 'product_name', 'product_price', 'product_amount', 'sales_price', 'discount', 'discount_total', 'real_receive_price']]

In [39]:
# data transform: need to transform the columns: sales_date, discount_total
df_alc_clean['sales_date'] = pd.to_datetime(df_alc_clean['sales_date'], format='%Y-%m-%d %H:%M:%S')
df_alc_clean['discount_total'] = df_alc_clean['discount_total'].astype(int)

/var/folders/r5/sw4x2qc12wz22xrjq85dxj940000gn/T/ipykernel_71534/1563608779.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alc_clean['sales_date'] = pd.to_datetime(df_alc_clean['sales_date'], format='%Y-%m-%d %H:%M:%S')
/var/folders/r5/sw4x2qc12wz22xrjq85dxj940000gn/T/ipykernel_71534/1563608779.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_alc_clean['discount_total'] = df_alc_clean['discount_total'].astype(int)


,sales_date,product_type,product_name,product_price,product_amount,sales_price,discount,discount_total,real_receive_price
0,2022-08-15 19:59:55,Other Classic,Dry Martini,320.0,1.0,320,0.0,0,320
8,2022-08-05 23:47:35,Beer,PERONI draft beer,250.0,1.0,250,0.0,0,250
9,2022-08-05 23:47:35,Beer,PERONI draft beer,250.0,1.0,250,0.0,0,250
11,2022-08-05 23:47:35,Signature Cocktail,龍柏,400.0,1.0,400,0.0,0,400
12,2022-08-05 23:47:35,Signature Cocktail,哈庫那瑪塔塔,420.0,1.0,420,0.0,0,420


In [48]:
# tryna see the pivot table of sales total
df_alc_clean.groupby('product_type')[['product_amount', 'sales_price', 'real_receive_price']].agg('sum').sort_values('sales_price', ascending=False)

,product_amount,sales_price,real_receive_price
product_type,,,
Signature Cocktail,1210.0,478270,475502
Classic Cocktail,237.0,82460,81530
Other Classic,161.0,57300,56880
Single Malt,49.0,55670,54360
Beer,196.0,36000,30317
Gin,59.0,25470,25470
Shot,204.0,15650,15450
House Wine,107.0,11900,11900
Cognac,4.0,5700,5700


In [51]:
df_alc_clean.to_csv('ctdh_alc_sales_aug_transform.csv', index=False)